In [ ]:
import tensorflow as tf
import keras
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, Input
from keras.models import Sequential, Model
from keras.layers import Layer
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import img_to_array, load_img


## Crawl data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

path = "/content/drive/MyDrive/data/Dataset/"
train, test = "Train", "Test"
train_path = path + train
test_path = path + test

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_data = train_datagen.flow_from_directory(train_path,
                                               target_size = (224, 224),
                                               batch_size = 32,
                                               class_mode = 'categorical')
test_data = test_datagen.flow_from_directory(test_path,
                                             target_size = (224, 224),
                                             batch_size = 32,
                                             class_mode = 'categorical')

Found 416 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


## Model and Training

In [ ]:
class feature_extractor(Layer):
  def __init__(self, input_shape):
    super(feature_extractor, self).__init__()
    self.conv1 = Conv2D(filters = 16, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu', input_shape = input_shape)
    self.maxpool1 = MaxPooling2D(pool_size = 2)
    self.batchnorm1 = BatchNormalization()

    self.conv2 = Conv2D(filters = 32, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu')
    self.maxpool2 = MaxPooling2D(pool_size = 2)
    self.batchnorm2 = BatchNormalization()

  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.batchnorm1(x)

    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.batchnorm2(x)
    return x

class my_model(Model):
  def __init__(self):
    super(my_model, self).__init__()
    self.extractor = feature_extractor(input_shape = (224, 224, 3))
    self.flatten = Flatten()
    self.fc2 = Dense(128, activation = 'relu')
    self.batchnorm = BatchNormalization()
    self.fc3 = Dense(2, activation = 'softmax')

  def call(self, x):
    x = self.extractor(x)
    x = self.flatten(x)
    x = self.fc2(x)
    x = self.batchnorm(x)
    x = self.fc3(x)
    return x


In [ ]:
model = my_model()
model.build((None, 224, 224, 3))
model.summary()
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = 'accuracy')


Model: "my_model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extractor_12 (feat  multiple                  5280      
 ure_extractor)                                                  
                                                                 
 flatten_10 (Flatten)        multiple                  0         
                                                                 
 dense_20 (Dense)            multiple                  11944064  
                                                                 
 batch_normalization_36 (Ba  multiple                  512       
 tchNormalization)                                               
                                                                 
 dense_21 (Dense)            multiple                  258       
                                                                 
Total params: 11950114 (45.59 MB)
Trainable params: 119

## Callback

In [ ]:
from keras.callbacks import Callback, CSVLogger, EarlyStopping

class LossCallback(Callback):
  def on_epoch_end(self, epoch, logs):
    print("\n Epoch {}, loss is {}".format(epoch, logs['loss']))

  def on_batch_end(self, batch, logs):
     print("\n Batch {}, loss is {}".format(batch, logs))

## CSVLogger

In [ ]:
csv_callback = CSVLogger(
    filename = 'logs.csv',
    separator = ',',
    append = False
)

## Early Stopping

In [ ]:
stopping = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.1,
    patience = 3
)

In [ ]:
model.fit(train_data, batch_size = 32, epochs = 3, verbose = 1, validation_data = test_data, callbacks = [csv_callback])

Epoch 1/3
13/13 [==============================] - 30s 2s/step - loss: 0.0129 - accuracy: 0.9976 - val_loss: 2.0204 - val_accuracy: 0.5000
Epoch 2/3
13/13 [==============================] - 30s 2s/step - loss: 0.0212 - accuracy: 0.9952 - val_loss: 1.5320 - val_accuracy: 0.5672
Epoch 3/3
13/13 [==============================] - 30s 2s/step - loss: 0.0089 - accuracy: 0.9976 - val_loss: 2.2586 - val_accuracy: 0.4478
